In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


In [3]:
# df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['headline', 'stock', 'headline_cleaned']]
df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[:1000]


In [4]:
df.head()

,id,headline,url,publisher,date,stock,headline_cleaned,senti_pos_score,senti_neg_score,senti_score,open,close
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",0.125,0.000,0.125,92.13,90.38
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",0.125,0.000,0.125,90.65,90.49
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",0.250,0.125,0.125,86.23,86.13
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",0.000,0.000,0.000,85.00,84.98
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22,A,"['security.n.01', 'neutral.n.01', 'technology....",0.500,0.250,0.250,85.00,84.98


In [5]:
def stockDevelopement(row):
    stock_differenz = row['open'] - row['close']
    quotient = row['close']/row['open']
    if quotient < 1:
        return -1
    elif quotient > 1:
        return 1
    else: return 0

In [6]:
df['stockPriceChange'] = df.apply(stockDevelopement, axis=1)

In [7]:
df = df[['id','headline', 'stock', 'headline_cleaned','open','close', "stockPriceChange"]]
df = df.rename(columns={'headline': '_headline', 'headline_cleaned':'_headline_cleaned', 'stock': '_stock', 'open': '_open', 'close':'_close', 'stockPriceChange':'_stockPriceChange'})

In [8]:
def tokenz_in_sentence(hl):
    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    sentence = ' '.join(word_list)
    return sentence

In [9]:
df['_headline_sentence'] = df._headline_cleaned.apply(tokenz_in_sentence)

In [10]:
df = df.set_index('id')
df.head(300)

,_headline,_stock,_headline_cleaned,_open,_close,_stockPriceChange,_headline_sentence
id,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.13,90.38,-1,stock hit high friday
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.65,90.49,-1,stock hit high wednesday
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,-1,large mover friday
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.00,84.98,-1,stock travel friday session
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,-1,security neutral technology raise monetary_val...
...,...,...,...,...,...,...,...
4801,Unknown Airport Group Says US Airports Will Ne...,AAL,"['unknown.a.01', 'airport.n.01', 'group.n.01',...",9.58,9.25,-1,unknown airport group state airport need least...
4802,Shares of several airline companies are tradin...,AAL,"['share.n.01', 'several.s.01', 'airline.n.01',...",9.58,9.25,-1,share several airline company trade low contin...
4803,El Al Will Extend Its Hald On All Its Schedule...,AAL,"['flight.n.01', 'israel.n.01', 'may.n.01', 'ca...",10.26,9.51,-1,flight israel may cargo flight one-off service


In [11]:
splitted_set = train_test_split(df, test_size=0.3)
train_set_un = splitted_set[0]
test_set_un = splitted_set[1]
train_set = train_set_un['_headline_sentence'].tolist()
test_set = test_set_un['_headline_sentence'].tolist()

In [12]:
train_set_un.shape

(700, 7)

In [13]:
count_vectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

In [14]:
count_wm = count_vectorizer.fit_transform(train_set)
tfidf_wm = tfidf_vectorizer.fit_transform(train_set)

In [15]:
count_tokenz = count_vectorizer.get_feature_names()
tfidf_tokenz = tfidf_vectorizer.get_feature_names()

In [16]:
# df_countvect = pd.DataFrame(data = count_wm.toarray(), columns=count_tokenz)
df_tfidfvect = pd.DataFrame(data= tfidf_wm.toarray(), index=train_set_un.index, columns=tfidf_tokenz)

In [17]:
df_tfidfvect = df_tfidfvect.sort_index(ascending=True)

In [18]:
df_tfidfvect.head()

,aaron,ability,able,acadia,ache,acquiring,acquisition,action,activity,ad,...,work,worker,world,worry,worsen,worth,write,xerox,year,yesterday
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df_tfidfvect = df_tfidfvect[:1000]

In [19]:
cols_to_delete = []
for column in df_tfidfvect:
    
    # print(df_tfidfvect[df_tfidfvect[column] > 0].shape[0])
    if df_tfidfvect[df_tfidfvect[column] > 0].shape[0] < 70:
        cols_to_delete.append(column)


In [20]:
print(len(cols_to_delete))
print(df_tfidfvect.shape[1])


1070
1080


In [21]:
df_tfidfvect.drop(axis=1, columns=cols_to_delete, inplace=True)



In [22]:
df_tfidfvect.head()
# df_tfidfvect.shape

,airline,american,apple,company,high,monetary_value,share,stock,target,trade
id,,,,,,,,,,
0,0.0,0.0,0.0,0.000000,0.476389,0.000000,0.000000,0.377622,0.000000,0.000000
1,0.0,0.0,0.0,0.000000,0.450588,0.000000,0.000000,0.357171,0.000000,0.000000
5,0.0,0.0,0.0,0.000000,0.000000,0.330488,0.000000,0.000000,0.336795,0.000000
6,0.0,0.0,0.0,0.000000,0.000000,0.344319,0.000000,0.000000,0.350891,0.000000
7,0.0,0.0,0.0,0.314331,0.336078,0.000000,0.283671,0.000000,0.000000,0.299343


In [23]:
df.head()
# df.shape

,_headline,_stock,_headline_cleaned,_open,_close,_stockPriceChange,_headline_sentence
id,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'frida...",92.13,90.38,-1,stock hit high friday
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wedne...",90.65,90.49,-1,stock hit high wednesday
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,-1,large mover friday
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', '...",85.00,84.98,-1,stock travel friday session
4,B of A Securities Maintains Neutral on Agilent...,A,"['security.n.01', 'neutral.n.01', 'technology....",85.00,84.98,-1,security neutral technology raise monetary_val...


In [24]:
# merged_df = pd.concat([df.set_index('id'), df_tfidfvect.set_index('id')], axis=1, join='inner')
mergred = df.merge(df_tfidfvect, on='id')

In [26]:
mergred.sample(5)

,_headline,_stock,_headline_cleaned,_open,_close,_stockPriceChange,_headline_sentence,airline,american,apple,company,high,monetary_value,share,stock,target,trade
id,,,,,,,,,,,,,,,,,
4903,Boeing Recommends Airlines To Inspect The 737 ...,AAL,"['airline.n.01', 'inspect.v.01', 'soap.n.03', ...",28.8400,28.3300,-1,airline inspect soap airplane year,0.242558,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
4857,Bank of America Maintains Underperform on Amer...,AAL,"['bank.n.01', 'united_states.n.01', 'american....",16.0600,15.6600,-1,bank united_states american airline group rais...,0.263213,0.291378,0.000000,0.000000,0.000000,0.254952,0.000000,0.0,0.259818,0.00000
6182,AAON shares are trading higher after the compa...,AAON,"['share.n.01', 'trade.v.01', 'high.a.01', 'com...",52.4200,57.5900,1,share trade high company report sale,0.000000,0.000000,0.000000,0.391183,0.418246,0.000000,0.353026,0.0,0.000000,0.37253
7378,"Halliburton, United, Among Key Names Reporting...",AAPL,"['cardinal.s.01', 'name.n.01', 'report.v.01', ...",69.4875,69.2325,-1,cardinal name report consequence airline due s...,0.234643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000
7448,Apple Tells Staff U.S. Stores to Remain Closed...,AAPL,"['apple.n.01', 'tell.n.01', 'staff.n.01', 'uni...",60.7000,60.3525,-1,apple tell staff united_states_government shop...,0.000000,0.000000,0.189354,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000


In [27]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

X = mergred.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', '_stockPriceChange', '_headline_sentence'])
Y = mergred['_stockPriceChange']

In [43]:
naive_bayes_classifier = MultinomialNB(
)
naive_bayes_classifier.fit(X, Y)

MultinomialNB()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 300)